### Tratamento de CSV para Banco de Dados Relacional (MySQL)

Utilizei este notebook para tratar os dados do CSV e inserir no banco de dados relacional MySQL.

In [62]:
import pandas as pd
import json

#### Importando os dados

In [74]:
df_jogadores = pd.read_csv('../../data/dados/players_fifa22.csv', sep = ';', low_memory = False)
df_clubes = pd.read_csv('../../data/dados/teams_fifa22.csv', sep = ',', low_memory = False)


with open('../../data/dados/country_league.json', 'r') as arquivo:
    linha = arquivo.read()
    pais_liga = json.loads(linha)

df_pais_liga = pd.DataFrame.from_dict(pais_liga, orient='index')

#### Iniciando alguns dataframes para auxiliar no tratamento dos dados

In [67]:
paises = df_jogadores['Nationality'].drop_duplicates().sort_values(ascending = True)
posicoes = df_jogadores['BestPosition'].drop_duplicates()
habilidades = df_jogadores.columns[29:69]


#### Limpando o dataset de jogadores (Entidade)

In [57]:
# Função pra dropar colunas que não serão utilizadas
df_jogadores.drop(habilidades, axis = 1, inplace = True)

# O ID será usado para simular o ID autoincrementado do banco de dados
df_jogadores['ID'] = range(1, len(df_jogadores) + 1)

#### Criando os datasets das entidades fortes

In [58]:
df_nacionalidade = pd.DataFrame({'ID': range(1, len(paises) + 1), 'nome': paises.values})
df_habilidade = pd.DataFrame({'ID': range(1, len(habilidades) + 1), 'nome': habilidades.values})
df_posicao = pd.DataFrame({'ID': range(1, len(posicoes) + 1), 'nome': posicoes.values})

#### Limpando dataframes com colunas excedentes para suas respectivas tabelas

In [59]:
df_liga = df_clubes.drop(['ID', 'Name', 'LeagueId', 'Overall', 'Attack', 'Midfield', 'Defence', 'Players', 'StartingAverageAge', 'AllTeamAverageAge', 'TransferBudget',	'DomesticPrestige', 'IntPrestige'], axis = 1)
df_liga.drop_duplicates(inplace = True)
df_liga.drop(df_liga[df_liga['League'] == '1024'].index, inplace = True)

df_clube = df_clubes.drop(['ID', 'LeagueId', 'Overall', 'Attack', 'Midfield', 'Defence', 'Players', 'StartingAverageAge', 'AllTeamAverageAge'], axis = 1)

# O dataframe df_jogador receberá os dados da tabela de jogador, enquanto o dataframe df_jogador manterá salvo os relacionamento com outras entidades para uso futuro
df_jogador = df_jogadores.drop(['ID', 'Nationality', 'Growth', 'TotalStats', 'BaseStats', 'Positions', 'BestPosition', 'Club', 'ClubPosition', 'ClubNumber', 'NationalTeam', 'NationalPosition', 'NationalNumber', 'AttackingWorkRate', 'DefensiveWorkRate'], axis = 1)

#### Removendo possibilidade de seleções

In [60]:
df_liga.drop(df_liga[df_liga['League'].str.contains('International')].index, inplace = True)
df_clube.drop(df_clube[df_clube['League'].str.contains('International')].index, inplace = True)

#### Criando e alterando dados para complementar o banco de dados

In [70]:
pos_tuplas = [('CAM', 'MAT', 'Center attacking midfielder', 'Meia-atacante'),
('CB', 'ZAG', 'Counter back', 'Zagueiro'),
('CDM', 'VOL', 'Central defensive midifielder', 'Volante'),
('CF', 'CEN', 'Center Forward', 'Centroavante'),
('CM', 'MC', 'Central Midfielder', 'Meio campista central'),
('GK', 'GOL', 'Goalkeeper', 'Goleiro'),
('LB', 'LE', 'Left back', 'Lateral esquerdo'),
('LF', 'ATA', 'Striker', 'Atacante'),
('LM', 'ME', 'Left Midfielder', 'Meia-esquerda'),
('LW', 'PE', 'Left wing', 'Ponta-esquerda'),
('LWB', 'AE', 'Lft wing back', 'Ala esquerdo'),
('RB', 'LD', 'Right back', 'Lateral direito'),
('RES', 'RES', 'Reserve', 'Reserva'),
('RF', 'ATA', 'Striker', 'Atacante'),
('RM', 'MD', 'Right midfielder', 'Meia-direita'),
('RW', 'PD', 'Right wing', 'Ponta-direita'),
('RWB', 'AD', 'Right wing back', 'Ala direito'),
('ST', 'ATA', 'Striker', 'Atacante'),
('SUB', 'SUB', 'Substitute', 'Substituto')]

In [27]:
df_clube['ID'] = range(1, len(df_clube) + 1)

df_liga['ID'] = range(1, len(df_liga) + 1)
df_jogador['ID'] = range(1, len(df_jogador) + 1)

df_pais = pd.DataFrame({'ID': range(1, len(paises) + 1), 'nome': paises.values})

df_posicao = pd.DataFrame(pos_tuplas, columns = ['nome', 'abreviacao', 'descricao', 'descricao_pt'])
df_posicao['ID'] = range(1, len(df_posicao) + 1)

#### Alterando os dataframes para que abriguem os relacionamentos entre as tabelas

In [37]:
df_clube['liga_id'] = df_clube['League'].apply(lambda x: df_liga[df_liga['League'] == x]['ID'].values[0])
df_clube.drop('League', axis = 1, inplace = True)

In [88]:
df_jogador['pais_id'] = df_jogadores['Nationality'].apply(lambda x: df_pais[df_pais['nome'] == x]['ID'].values[0])
df_jogador.drop('Nationality', axis = 1, inplace = True)

KeyError: "['Nationality'] not found in axis"

,Name,FullName,Age,Height,Weight,Overall,Potential,ValueEUR,WageEUR,ReleaseClause,ContractUntil,ClubJoined,OnLoad,PreferredFoot,pais_id
0,L. Messi,Lionel Messi,34,170,72,93,93,78000000,320000,144300000,2023.0,2021,False,Left,7
1,R. Lewandowski,Robert Lewandowski,32,185,81,92,92,119500000,270000,197200000,2023.0,2014,False,Right,124
2,K. Mbappé,Kylian Mbappé,22,182,73,91,95,194000000,230000,373500000,2022.0,2018,False,Right,57
3,J. Oblak,Jan Oblak,28,188,87,91,93,112000000,130000,238000000,2023.0,2014,False,Right,138
4,K. De Bruyne,Kevin De Bruyne,30,181,70,91,91,125500000,350000,232200000,2025.0,2015,False,Right,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19243,A. Shaikh,Arsh Shaikh,18,186,74,47,67,110000,500,347000,2022.0,2020,False,Right,76
19244,N. Saliba,Nathan-Dylan Saliba,17,173,71,47,69,150000,500,368000,2021.0,2020,False,Right,26
19245,M. O'Brien,Matthew O'Brien,18,176,71,47,61,110000,500,239000,2021.0,2020,False,Right,127
19246,Wang Shilong,Shilong Wang,20,177,64,47,60,110000,2000,245000,2022.0,2020,False,Right,31


,ID,nome
0,1,Afghanistan
1,2,Albania
2,3,Algeria
3,4,Andorra
4,5,Angola
5,6,Antigua and Barbuda
6,7,Argentina
7,8,Armenia
8,9,Australia
9,10,Austria
